### set seed so that the model outputs are reproducible
After the kernel is restarted the same results are created.

In [1]:
import keras
import tensorflow as tf
from tensorflow.keras import initializers
# Set the seed using keras.utils.set_random_seed. This will set:
# 1) `numpy` seed
# 2) `tensorflow` random seed
# 3) `python` random seed
seed = 0
keras.utils.set_random_seed(seed)

# This will make TensorFlow ops as deterministic as possible, but it will
# affect the overall performance, so it's not enabled by default.
# `enable_op_determinism()` is introduced in TensorFlow 2.9.
tf.config.experimental.enable_op_determinism()

## Load the data

In [2]:
import pandas as pd
import numpy as np
import os
import CRPS.CRPS as pscore
import copy
from joblib import dump, load
from time import sleep
from tqdm import tqdm
import warnings


def check_Actuals(country_id, dataindex):
    # Check if the country_id exists in actual dataset
    if country_id not in country_actual_group_list[dataindex].groups.keys():
        return False
    else:
        return True

# check if the last month of a country in the feature dataset is 3 months before the first month that has to be predicted
def check_last_featureMonth(country_id, dataindex):
    # Check if the country_id exists in actual dataset
    if country_id not in country_actual_group_list[dataindex].groups.keys():
        raise ValueError('country does not have actuals')


    # last month of the feature dataset
    last_feature_month = country_feature_group_list[dataindex].get_group(country_id).index.get_level_values('month_id').unique().tolist()[-1]

    # first month of the actual dataset
    first_actual_month = country_actual_group_list[dataindex].get_group(country_id).index.get_level_values('month_id').unique().tolist()[0]

    # if the last month of the feature dataset in the country does not match the first of the actuals return false
    if (first_actual_month - 3) != last_feature_month:
        return False
    else:
        return True


# create the feature- and actuals-data list
# set the feature and actuals year lists
feature_years = ['2017','2018','2019','2020']
actual_years = ['2018','2019','2020','2021']

actuals_df_list = []
features_df_list = []

# path to the current directory
current_dir = os.getcwd()

for i in range(len(feature_years)):
    # relative paths to the parquet files
    relative_path_features = os.path.join('..', 'data', 'cm_features_to_oct' + feature_years[i] + '.parquet')
    relative_path_actuals = os.path.join('..', 'data', 'cm_actuals_' + actual_years[i] + '.parquet')

    path_features = os.path.join(current_dir, relative_path_features)
    path_actuals = os.path.join(current_dir, relative_path_actuals)

    # append datasets to the lists
    actuals_df_list.append({'year':actual_years[i], 'data':pd.read_parquet(path_actuals, engine='pyarrow')})
    features_df_list.append({'year':feature_years[i], 'data':pd.read_parquet(path_features, engine='pyarrow')})

# concat the feature datasets, so that every data contains the observations starting with january 1990
for i in range(1,len(features_df_list)):
    features_df_list[i]['data'] = pd.concat([features_df_list[i-1]['data'], features_df_list[i]['data']])



### Drop features that contain missing values

In [3]:
from sklearn.feature_selection import VarianceThreshold
from statsmodels.stats.outliers_influence import variance_inflation_factor


data = features_df_list[-1]['data']
if 'gleditsch_ward' in data.columns:
    data = data.drop(columns='gleditsch_ward') # column not necessary

## Features without missing values
columns_without_missing_values = data.columns[data.notna().all()]

for i in range(len(features_df_list)):
    data_set = features_df_list[i]['data']
    features_df_list[i]['data'] = data_set[columns_without_missing_values]

# last dataset contains all other datasets (because of concat) 
all_features = features_df_list[-1]['data'].columns


### Feature subsets

In [4]:
## different feature selection from views
# 59 features that map the conflict history of a country
conflict_history = [
    'ged_sb', 'decay_ged_sb_5', 'decay_ged_os_5',
    'splag_1_decay_ged_sb_5', 'wdi_sp_pop_totl', 'ged_sb_tlag_1',
    'ged_sb_tlag_2', 'ged_sb_tlag_3', 'ged_sb_tlag_4',
    'ged_sb_tlag_5', 'ged_sb_tlag_6', 'ged_sb_tsum_24',
    'decay_ged_sb_100', 'decay_ged_sb_500', 'decay_ged_os_100',
    'decay_ged_ns_5', 'decay_ged_ns_100', 'ged_ns', 'ged_os',
    'acled_sb', 'acled_sb_count', 'acled_os',
    'ged_os_tlag_1', 'decay_acled_sb_5', 'decay_acled_os_5',
    'decay_acled_ns_5', 'splag_1_decay_ged_os_5',
    'splag_1_decay_ged_ns_5'
]


# 59 features that are drawn from the Varieties of Democracy project
vdem = [
    'ged_sb', 'decay_ged_sb_5', 'decay_ged_os_5',
    'splag_1_decay_ged_sb_5', 'wdi_sp_pop_totl', 'vdem_v2x_delibdem',
    'vdem_v2x_egaldem', 'vdem_v2x_libdem', 'vdem_v2x_libdem_48',
    'vdem_v2x_partip', 'vdem_v2x_accountability',
    'vdem_v2x_civlib', 'vdem_v2x_clphy', 'vdem_v2x_cspart',
    'vdem_v2x_divparctrl', 'vdem_v2x_edcomp_thick', 'vdem_v2x_egal',
    'vdem_v2x_execorr', 'vdem_v2x_frassoc_thick', 'vdem_v2x_gencs',
    'vdem_v2x_gender', 'vdem_v2x_genpp', 'vdem_v2x_horacc',
    'vdem_v2x_neopat', 'vdem_v2x_pubcorr', 'vdem_v2x_rule',
    'vdem_v2x_veracc', 'vdem_v2x_freexp', 'vdem_v2xcl_acjst', 
    'vdem_v2xcl_dmove', 'vdem_v2xcl_prpty', 'vdem_v2xcl_rol', 
    'vdem_v2xcl_slave', 'vdem_v2xdl_delib', 'vdem_v2xeg_eqdr',
    'vdem_v2xeg_eqprotec', 'vdem_v2xel_frefair', 'vdem_v2xel_regelec',
    'vdem_v2xme_altinf', 'vdem_v2xnp_client', 'vdem_v2xnp_regcorr',
    'vdem_v2xpe_exlecon', 'vdem_v2xpe_exlpol', 'vdem_v2xpe_exlgeo',
    'vdem_v2xpe_exlgender', 'vdem_v2xpe_exlsocgr', 'vdem_v2xps_party',
    'vdem_v2xcs_ccsi', 'vdem_v2xnp_pres', 'vdem_v2xeg_eqaccess',
    'vdem_v2x_diagacc', 'vdem_v2clrgunev', 'splag_vdem_v2x_libdem',
    'splag_vdem_v2xcl_dmove', 'splag_vdem_v2x_accountability',
    'splag_vdem_v2xpe_exlsocgr', 'splag_vdem_v2xcl_rol', 'wdi_sm_pop_netm',
    'wdi_sp_dyn_imrt_in'
]

# 30 features that are drawn from the WDI as well as some conflict history indicators
wdi = [
    'ged_sb', 'decay_ged_sb_5', 'decay_ged_os_5',
    'splag_1_decay_ged_sb_5', 'wdi_sp_pop_totl', 'wdi_ag_lnd_frst_k2',
    'wdi_dt_oda_odat_pc_zs', 'wdi_ms_mil_xpnd_gd_zs', 'wdi_ms_mil_xpnd_zs',
    'wdi_nv_agr_totl_kd', 'wdi_nv_agr_totl_kn', 'wdi_ny_gdp_pcap_kd',
    'wdi_sp_dyn_le00_in', 'wdi_se_prm_nenr', 'wdi_sh_sta_maln_zs', 
    'wdi_sh_sta_stnt_zs', 'wdi_sl_tlf_totl_fe_zs', 'wdi_sm_pop_refg_or', 
    'wdi_sm_pop_netm', 'wdi_sm_pop_totl_zs', 'wdi_sp_dyn_imrt_in', 
    'wdi_sh_dyn_mort_fe', 'wdi_sp_pop_1564_fe_zs', 'wdi_sp_pop_65up_fe_zs',
    'wdi_sp_pop_grow', 'wdi_sp_urb_totl_in_zs',
    'splag_wdi_sl_tlf_totl_fe_zs', 'splag_wdi_sm_pop_refg_or',
    'splag_wdi_sm_pop_netm', 'splag_wdi_ag_lnd_frst_k2'
]

ged = ['ged_sb']

feature_subset_dict = {'conflict_history':conflict_history,
                       'vdem':vdem,
                       'wdi':wdi,
                       'all':all_features,
                       'ged':ged}

### Group data by country_id

In [5]:
country_list = sorted(features_df_list[3]['data'].index.get_level_values('country_id').unique().tolist())

# country group list of all four datasets
country_feature_group_list = []
country_actual_group_list = []
# fill list 
for i in range(len(features_df_list)):
    country_feature_group_list.append(features_df_list[i]['data'].groupby('country_id'))
    country_actual_group_list.append(actuals_df_list[i]['data'].groupby('country_id'))

### List of the countries for which a prediction is requested

In [6]:
relative_path_countrylist = os.path.join('..', 'data', 'country_list.csv')
path_countrylist = os.path.join(current_dir, relative_path_countrylist)

# CSV-Datei einlesen und als Pandas-Datensatz speichern
countryList_prediction = pd.read_csv(path_countrylist)
country_list_views = countryList_prediction.loc[:,'country_id'].values.tolist() 

month_list = []
countries_to_remove = []
for country_id in country_list:

    if country_id in country_list_views:
        feature_data_views = country_feature_group_list[0].get_group(country_id)

        # numbers of months from the feature dataset
        month_list_feature_data_original = feature_data_views.index.get_level_values('month_id').tolist()
        number_months_feature_data = len(month_list_feature_data_original) 

        # check if actuals exist for the country
        if check_Actuals(country_id, 0):
            # check if the last feature month is 3 months before the first actuals month
            if not check_last_featureMonth(country_id, 0): 
                month_list.append([str(country_id) +' last month missing'])
            else:
                month_list.append([number_months_feature_data, country_id])
        else:
            month_list.append(str(country_id) + ' no actuals')
    else:
        countries_to_remove.append(country_id)

country_list = list(set(country_list) - set(countries_to_remove))
month_list.sort()

## Feed Forward Neural Network
Goal is to estimate the empirical distribution of the fatalities per month.
### Definition of the CRPS loss function and the Feed forward Neural Network subclass

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer, Input, Dense, Lambda, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Loss
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.callbacks import EarlyStopping

# crps loss function 
def crps(y_true, S):
    """
    Computes continuous ranked probability score:

    Parameters
    ----------
    y_true : tf tensor of shape (BATCH_SIZE, 1)
        True values.
    S : tf tensor of shape (BATCH_SIZE, N_SAMPLES)
        Predictive samples.

    Returns
    -------
    tf tensor of shape (BATCH_SIZE,)
        Scores.

    """
    beta=1
    n_samples = S.shape[-1]

    def expected_dist(diff, beta):
        return K.sum(K.pow(K.sqrt(K.square(diff)+K.epsilon()), beta),axis=-1) #axis = -1: last dimension <=> N_SAMPLES
    es_1 = expected_dist(y_true - S, beta)
    es_2 = 0
    for i in range(n_samples):
        es_2 = es_2 + expected_dist(K.expand_dims(S[:,i]) - S, beta)
    return es_1/n_samples - es_2/(2*n_samples**2)


class CRPSLoss(Loss):
    def call(self, y_true, S):
        return crps(y_true, S)

# Define custom ReLU activation function
class ReLUTransform(Layer):
    def call(self, inputs):
        return tf.nn.relu(inputs)

# number of neurons per hidden layer (equally spaced)
def get_numberNeurons_per_hiddenlayer(numberHiddenLayers, numberNeurons):

    neurons_per_HL = np.round(numberNeurons/numberHiddenLayers)

    if neurons_per_HL <= 0:
        raise ValueError('Number of neurons per hidden layer less than one.')

    return neurons_per_HL

# Define the Feed Forward Neural Network subclass
class FeedForwardNN(tf.keras.Model):
    def __init__(self, input_shape, name="FeedFwdNN", neurons_output = 200, number_hidden_layers=1, 
                 number_neurons=10, dropout_rate=None):
        super(FeedForwardNN, self).__init__(name=name)

        number_neurons_per_hlayer = get_numberNeurons_per_hiddenlayer(number_hidden_layers, number_neurons)

        self.hidden_layers = []

        self.hidden_layers.append(Dense(number_neurons_per_hlayer, activation='relu'))
        if dropout_rate is not None:
            self.hidden_layers.append(Dropout(dropout_rate))

        if number_hidden_layers > 1:
            for _ in range(number_hidden_layers - 1):
                self.hidden_layers.append(Dense(number_neurons_per_hlayer, activation='relu'))

        self.untransformed_output = Dense(neurons_output)
        self.final_output = Lambda(ReLUTransform())

        self.model = self.build(input_shape)
        
    def call(self, inputs):
        x = inputs
        for layer in self.hidden_layers:
            x = layer(x)
        x = self.untransformed_output(x)
        y = self.final_output(x)
        return y

### Definition of the functions to perform the train-/test-split with rolling windows

In [8]:
from sklearn import preprocessing

## conflict trap
# drops all months before a starting conflict
# defintion of a beginning conflict: fatalities(monthX) > 0 with mean(fatalities(window_size number months starting with monthX)) > threshold
# (average fatalities per month in the starting half year are greater than the threshold)
# iterates trough the dataset beginning with the first entry
def drop_before_conflict_trap(data, threshold, window_size, minimal_data_size):
    index_ged_sb = data.columns.get_loc('ged_sb')

    start_index = 0
    while start_index < len(data.iloc[:,index_ged_sb]) - window_size + 1:
        window = data.iloc[start_index:start_index + window_size, index_ged_sb].to_list()
        
        if window[0] > 0 and sum(window) / window_size >= threshold:
            break
        else:
            start_index += 1


    if len(data) >= minimal_data_size:
        # if there is no conflict trap do nothing
        if start_index == len(data.iloc[:,index_ged_sb]) - window_size + 1:
            return data
        # if the truncation would result in a too small dataset prevent this
        elif len(data.iloc[start_index:, :]) < minimal_data_size:
            return data.iloc[-minimal_data_size:, :]
        # drop every entry before the conflict trap 
        else:
            return data.iloc[start_index:, :]
    else:
        return data
    

## function used to calculate w_max, number of rolling windows etc.
# length of a whole window (containing w input months and 12 acutal months)
def rollingWindowLength(w):
    return w + 2 + 12

# number of months available for training (after removing the test months)
def number_train_months(numberMonths_available, w):
    #  all months feature data   -  test set input
    return numberMonths_available - w

def number_rolling_windows(numberMonths_available, w):
    return max(0,numberMonths_available - rollingWindowLength(w) + 1)


def find_max_W(numberMonths_available, w_min, w_max):
    if number_rolling_windows(numberMonths_available, w_min) == 0:
        raise ValueError('not enough months for one training window with w_min = ' + str(w_min))

    # find the maximal w
    max_W = w_max
    number_months_train = number_train_months(numberMonths_available, max_W)
    number_train_rollwindows_wmax = number_rolling_windows(number_months_train, max_W)

    # calculate w_max so that the number of rolling windows for the validation set is >= 1
    # and that
    # the number of rolling windows for the train set is >= 1
    while number_train_rollwindows_wmax == 0 and max_W > w_min:
        max_W -= 1
        number_months_train = number_train_months(numberMonths_available, max_W)
        number_train_rollwindows_wmax = number_rolling_windows(number_months_train, max_W)

    return max_W


def month_lists_TrainTest(w_min, w_max, month_list_feature_data):
    # numbers of months from the shortened feature dataset
    number_months = len(month_list_feature_data)
    
    # find w_max (as mentioned above, if there are not enoug months, the w_max has to be < w_max)
    w_max_local = find_max_W(number_months, w_min, w_max)

    w = w_max_local

    # length of the maximum rolling window and the used "unreal" acutals starting 3 months after the last used month
    n_train_months = number_train_months(number_months, w)

    month_list_train = month_list_feature_data[0:n_train_months]
    month_list_test = month_list_feature_data[-w:]

    return month_list_train, month_list_test, w


def Train_ArrayXY_split(w, month_list, data_feature, s):
    X = []
    Y = []

    train_months = len(month_list)

    number_rolling_windows_train = number_rolling_windows(train_months, w)

    for i in range(0, number_rolling_windows_train):
        starting_month_features = month_list[i]

        index_ending_month_features = i + w - 1
        ending_month_features = month_list[index_ending_month_features]

        starting_month_unrActuals = month_list[index_ending_month_features + 3]
        ending_month_unrActuals = month_list[index_ending_month_features + 14]

        window_features = data_feature.loc[slice(starting_month_features, ending_month_features), :] # excluding "unreal" actuals
        window_actuals = data_feature.loc[slice(starting_month_unrActuals, ending_month_unrActuals), 'ged_sb'].iloc[s - 3] # "unreal" actuals


        normalized_window_features = preprocessing.normalize(window_features)
        window_features_array = np.array([normalized_window_features.flatten()])[0]

        window_actual_array = np.array([window_actuals])

        X.append(window_features_array)
        Y.append(window_actual_array)

    X = np.array(X)
    Y = np.array(Y)

    return X,Y

def Test_ArrayXY_split(month_list, data_feature, data_actual, s):
    X = []
    Y = []

    starting_month_test = month_list[0]
    ending_month_test = month_list[-1]

    window_features_test = data_feature.loc[slice(starting_month_test, ending_month_test), :] # all w features to predict the fatalities
    window_actuals_test = data_actual.iloc[s - 3].values # real actuals

    normalized_window_features_test = preprocessing.normalize(window_features_test)
    window_features_array_test = np.array([normalized_window_features_test.flatten()])[0]

    window_actual_array_test = window_actuals_test

    X.append(window_features_array_test)
    Y.append(window_actual_array_test)

    X = np.array(X)
    Y = np.array(Y)

    return X,Y

## Estimation of the future fatalites
### Set the year to predict

In [9]:
prediction_year = '2021' # 2019, 2020, 2021
dataset_index = actual_years.index(prediction_year)

### Manipulate country list for the prediction

In [10]:
zero_fatalities_country_list = []
countries_with_high_percentage_list = []
someNonzero_fatalities_country_list = []

for country_id in country_list:
    feature_data_all = country_feature_group_list[dataset_index].get_group(country_id)

    # Berechnen des Prozentsatzes der Werte größer als 0 in der Spalte 'ged_sb'
    positive_percentage = (feature_data_all['ged_sb'] > 0).mean() * 100

    if (feature_data_all['ged_sb'] == 0).all():
        zero_fatalities_country_list.append(country_id)
    elif positive_percentage >= 60:
        countries_with_high_percentage_list.append(country_id)
    else:
        someNonzero_fatalities_country_list.append(country_id)

### Prediction

In [11]:
import random
from random import randint, uniform

# this function is not needed in the prediction (only hyperparam tuning)
def draw_params(seed):

    #random.seed(seed)
    keras.utils.set_random_seed(seed)
    
    dropout = random.choice([0, 1])
    
    ### ranges of hyperparameters
    ranges = {
        'wmax': randint(1, 12),
        'numbHiddenL': randint(1, 6),
        'relNeurons': uniform(0.1, 1.0),
        'lr': uniform(0.001, 0.15), 
        'featureSubset': random.choice(list(feature_subset_dict.keys())),
        'batch_size': randint(1, 6),
        'dropoutrate': uniform(0.1, 0.5),
        'epochs': randint(3, 40)
    }
    
    if dropout == 0:
        ranges['dropoutrate'] = 0


    # Erstelle ein neues Dictionary, um die gezogenen Werte zu speichern
    drawn_params = {}

    # Ziehe die Werte und speichere sie im neuen Dictionary
    for param_name, param_value in ranges.items():
        if isinstance(param_value, float):
            if param_name == 'relNeurons' or param_name == 'dropoutrate':
                drawn_params[param_name] = round(param_value, 2)
            else:
                drawn_params[param_name] = round(param_value, 3)
        else:
            drawn_params[param_name] = param_value

    return drawn_params

def get_numberNeurons_all_layers(inputNeurons, outputNeurons, lam):
    upper_bound = max(inputNeurons, outputNeurons)
    lower_bound = min(inputNeurons, outputNeurons)

    return np.round(lam * (upper_bound - lower_bound) + lower_bound)


def evaluate_model(hyperParameters, s, feature_dataset, actuals_dataset):

    ## fixed paramters--------
    w_min= 1
    number_output_neurons = 200
    var_threshold = 0.05
    mean_fatlities_per_month_threshold = 5
    #--------------

    ## prepare feature dataset
    # only FEATURE SUBSET
    feature_data = feature_dataset.loc[:,feature_subset_dict[hyperParameters['featureSubset']]]

    ## Drop features with NEAR ZERO VARIANCE (but dont drop 'ged_sb' -> needed for conflict trap detection)
    columns_to_keep = [col for col in feature_data.columns if (col == 'ged_sb') or (feature_data[col].var() >= var_threshold)]
    feature_data = feature_data[columns_to_keep]

    ## remove months before the CONDFLICT TRAP (regime change)
    # if the average number of fatalities per month in 6 months is above 'mean_fatlities_per_month_threshold' and the fatalities of the starting month are > 0 
    # the conflict trap starts and all obsservations before that month are dropped
    # 76 is the minimal length of the dataframe (refers to the minimal size of the data for all countries -> country_id 246 len = 76) 
    feature_data = drop_before_conflict_trap(feature_data, mean_fatlities_per_month_threshold, 6, 76)
    
    month_list_feature_data = feature_data.index.get_level_values('month_id').tolist()


    ### data split
    month_list_train, month_list_test, w = month_lists_TrainTest(w_min, hyperParameters['wmax'], month_list_feature_data)

    ## training dataset------
    X_train, Y_train = Train_ArrayXY_split(w, month_list_train, feature_data, s)

    ## test dataset-------
    X_test, Y_test = Test_ArrayXY_split(month_list_test, feature_data, actuals_dataset, s)

    ### prediction with the neural net
    ## Define inputs with predefined shape
    input_shape = (len(X_train[0]),)
    inputs = Input(shape=input_shape)

    # only needed in hyperparameter tuning
    """ hyperParameters['numbNeurons'] = get_numberNeurons_all_layers(input_shape[0], 
                                                               number_output_neurons, 
                                                               hyperParameters['relNeurons']) """
    # overwrite the old w
    hyperParameters['w'] = w

    ## define neural net
    # Create an instance of the FeedForwardNN model
    nameString = 'FFwdNN_s' + str(s)
    model = FeedForwardNN(input_shape=inputs.shape, name=nameString, neurons_output = number_output_neurons,
                          number_hidden_layers=hyperParameters['numbHiddenL'], 
                          number_neurons=hyperParameters['numbNeurons'], 
                          dropout_rate=hyperParameters['dropoutrate'])

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=hyperParameters['lr']), loss=CRPSLoss())

    # fit the model
    history = model.fit(X_train, Y_train, 
                        batch_size=hyperParameters['batch_size'], epochs=hyperParameters['epochs'],
                        verbose=0, shuffle=False)

    ## prediction
    prediction = model.predict(X_test)
    # round the prediction values to integers
    empirical_distribution = np.round(np.sort(prediction[0])).astype(int)

    # save the training and validation loss to generate plots
    loss = history.history['loss']
    epochs = range(1, len(loss) + 1)

    y_true = Y_test[0][0]
    crps_prediction = pscore(empirical_distribution,y_true).compute()[0]

    return loss, epochs, y_true, crps_prediction, empirical_distribution

In [12]:
import os
from joblib import load
import numpy as np

#randomctr220wRand_200s8_30valid_2021  !!

user_dir = os.path.expanduser('~')
file_path = os.path.join(user_dir, 'iCloudDrive\Joblib BA\Hyperparam', 'randomctr220wRand_200s8_30valid_2021.joblib')
loaded_vars_rndHyperparam = load(file_path)

sorted_best_runs, country_to_plot, prediction_year, s = loaded_vars_rndHyperparam

In [13]:
### prediction
s_prediction_list = list(range(3, 15))
number_s = len(s_prediction_list)
number_countries = len(country_list)

pred_year_string = 'prediction_' + prediction_year

# list to save the predictions for each country
NNet_prediction_list = [{'country_id': country, pred_year_string: []} for country in country_list]

syria_2020_bestS8_hyperparams = sorted_best_runs[0]['random_params']

# loop through all countries
for country_index in tqdm(range(number_countries)):
    country = country_list[country_index]

    ## load datasets
    features = country_feature_group_list[dataset_index].get_group(country)
    actuals = country_actual_group_list[dataset_index].get_group(country)


    if country in zero_fatalities_country_list:

        distribution = np.array([0]*200)

        NNet_prediction_list[country_index][pred_year_string].append({'s':[None for _ in range(number_s)],
                                                                  'distribution':[distribution for _ in range(number_s)],
                                                                  'actual':[None for _ in range(number_s)],
                                                                  'CRPS':[None for _ in range(number_s)], 
                                                                  'loss':[None for _ in range(number_s)], 
                                                                  'epochs':None,
                                                                  'hyperparams':None})
    else:
        NNet_prediction_list[country_index][pred_year_string].append({'s':[None for _ in range(number_s)],
                                                                  'distribution':[None for _ in range(number_s)],
                                                                  'actual':[None for _ in range(number_s)],
                                                                  'CRPS':[None for _ in range(number_s)], 
                                                                  'loss':[None for _ in range(number_s)], 
                                                                  'epochs':None,
                                                                  'hyperparams':syria_2020_bestS8_hyperparams})

    for s in s_prediction_list:

        if country in zero_fatalities_country_list:

            model_y_true = actuals.iloc[s - 3].values[0]
            model_crps_prediction = pscore(distribution,model_y_true).compute()[0]

            NNet_prediction_list[country_index][pred_year_string][0]['s'][s-3] = s
            NNet_prediction_list[country_index][pred_year_string][0]['actual'][s-3] = model_y_true
            NNet_prediction_list[country_index][pred_year_string][0]['CRPS'][s-3] = model_crps_prediction

        else:
            hyper_params = syria_2020_bestS8_hyperparams
            model_loss, model_epochs, model_y_true, model_crps_prediction, distribution = evaluate_model(hyper_params, s, features, actuals)

            NNet_prediction_list[country_index][pred_year_string][0]['s'][s-3] = s
            NNet_prediction_list[country_index][pred_year_string][0]['distribution'][s-3] = distribution
            NNet_prediction_list[country_index][pred_year_string][0]['actual'][s-3] = model_y_true
            NNet_prediction_list[country_index][pred_year_string][0]['CRPS'][s-3] = model_crps_prediction
            NNet_prediction_list[country_index][pred_year_string][0]['loss'][s-3] = model_loss
            NNet_prediction_list[country_index][pred_year_string][0]['epochs'] = model_epochs


  0%|          | 0/191 [00:00<?, ?it/s]

1/1 [==============================] - 0s 125ms/step


  2%|▏         | 3/191 [07:14<7:34:09, 144.94s/it]

1/1 [==============================] - 0s 136ms/step


  4%|▎         | 7/191 [14:41<5:06:52, 100.07s/it] 

...

 99%|█████████▉| 189/191 [10:56:51<12:39, 379.98s/it]

1/1 [==============================] - 0s 128ms/step


 99%|█████████▉| 190/191 [11:03:43<06:28, 388.95s/it]

1/1 [==============================] - 0s 135ms/step


100%|██████████| 191/191 [11:10:47<00:00, 210.72s/it]


In [14]:
from joblib import dump, load

joblib_string = 'FinalTask2_NN_' + str(prediction_year) + '_Hyperparamctr2204all' + '.joblib'

dump([NNet_prediction_list, country_list, pred_year_string, seed, zero_fatalities_country_list],  joblib_string)

['FinalTask2_NN_2021_Hyperparamctr2204all.joblib']

In [15]:
crps_values_test = []
for country_data in NNet_prediction_list:
    if country_data['country_id'] not in zero_fatalities_country_list:
        country_crps_list = []
        for s in range(0,number_s):
            country_crps_list.append(country_data[pred_year_string][0]['CRPS'][s])
        crps_values_test.append(np.mean(country_crps_list))

mean_crps_test = np.mean(crps_values_test)


print('Mean CRPS = ' + str(mean_crps_test) + ' \\'+'\\')
print('')

Mean CRPS = 57.05825592447918 \\

